In [ ]:
# 相关库导入
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
plt.rcParams['font.sans-serif']=['SimHei']  # 防止中文出现乱码
plt.rcParams['axes.unicode_minus'] = False  # 防止中文出现乱码

In [ ]:
# 图像导入
#img = cv2.imread('Images/lena.jpeg',1)
#img = cv2.imread('Images/son.jpg',1)
#img = cv2.imread('Images/woman2.png',1)
#img = cv2.imread('Images/cameraman.jpg',1)
#img = cv2.imread('Images/girl6.jpg',1)
#img = cv2.imread('Images/girl7.jpg',1)     # 终极素材
img = cv2.imread('Images/mopi.jpeg')
#img = cv2.imread('Images/scene.jpg',1)
#img = cv2.imread('Images/building.jpg',1)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
cv2.imshow('src',img)
cv2.waitKey(0)

In [ ]:
# Gray World 色彩均衡 效果一般
def Gray_World(img):
    height = img.shape[0]
    width = img.shape[1]
    sumR = 0
    sumG = 0
    sumB = 0
    max = 0
    for i in range(0,height):
        for j in range(0,width):
            sumR = sumR + img[i,j,2]
            sumG = sumG + img[i,j,1]
            sumB = sumB + img[i,j,0]              
    avgR = sumR/(height*width)
    avgG = sumG/(height*width)
    avgB = sumB/(height*width)
    print(avgR,avgG,avgB)
    avgGray = (avgR+avgG+avgB)/3
    ar = avgGray/avgR
    ag = avgGray/avgG
    ab = avgGray/avgB
    for i in range(0,height):
        for j in range(0,width):
            img[i,j,2] = int(img[i,j,2]*ar)         # 对于图像中每个像素img[i,j] 调整其R、G、B分量 new Ｒ　＝　old Ｒ ＊ ａｒ
            img[i,j,1] = int(img[i,j,1]*ag)
            img[i,j,0] = int(img[i,j,0]*ab)
            if(img[i,j,2]>max):                     # max为RGB三个分量中的最大值
                max = img[i,j,2]
            if(img[i,j,1]>max):
                max = img[i,j,1]
            if(img[i,j,0]>max):
                max = img[i,j,0]
    factor = max/255                                 # 归一化处理
    print(factor)
    if(factor>1):
        for i in range(0,height):
            for j in range(0,width):
                img[i,j,2] = int(img[i,j,2]/factor)
                img[i,j,1] = int(img[i,j,1]/factor)
                img[i,j,0] = int(img[i,j,0]/factor)
    cv2.imshow('dst',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#Img1 = Gray_World(img)


In [ ]:
# 伽马校正

In [ ]:
# 基于参考白的算法
pixels = cv2.calcHist([img], [0], None, [256], [0, 256])            # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组



In [ ]:
#  改进的YCrCb转换公式 垃圾不好用
ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      # RGB->YCrCb  
(Y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图
B = img[:,:,0]                                     # 获取B通道
G = img[:,:,1]
R = img[:,:,2]
for i in range(imgInfo[0]):
    for j in range(imgInfo[1]):
        if(Y[i,j]<200):
            cr = int((B[i,j]-Y[i,j])*0.713)
            cb = int((B[i,j]-Y[i,j])*0.564)
        else:
            print(R[i,j],Y[i,j])
            cr = ((R[i,j]-Y[i,j])**2*0.713*((-5000/91)*(Y[i,j]-200)**(-2.0)+7))
            print("cr="+str(cr))
            cb = int(-1*(B[i,j]-Y[i,j])**2*0.564*(125*(Y[i,j]-200)**(-2.0)-3))
            print("cb="+str(cb))
img2 = (Y,cr,cb)
img3 =  cv2.cvtColor(img,cv2.COLOR_YCrCb2BGR)
cv2.imshow("IMG",img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 线性动态调整
def Dynamic_adjust_liner(img,th1,th2):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]
    dst = img.copy()
    for i in range(0,height):
        for j in range(0,width):
                if(img[i,j]<th1):
                    dst[i,j]= 0
                elif(img[i,j]<th2):
                    dst[i,j]= 255/(th2-th1)*(img[i,j]-th1)
                elif(img[i,j]>=th2):
                     dst[i,j] = 255
    cv2.imshow('dst',dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dst

dst = Dynamic_adjust_liner(img,100,150)

In [ ]:
# 肤色检测 基于YCrCb颜色空间的CrCb分量筛选
def YCrCb_CrCb(img):
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    height = imgInfo[0]
    width = imgInfo[1]
    dst = img.copy()
    for i in range(0,height):
        for j in range(0,width):
            if(cr[i,j]>135 and cr[i,j]<175 and cb[i,j]>77 and cb[i,j]<127):
                dst[i,j] = (255,255,255)
            else:
                dst[i,j] = (0,0,0)
                
    cv2.imshow('dst',dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return dst

YCrCb_CrCb(img)

In [ ]:
# 肤色检测 基于YCrCb颜色空间的Cr分量+Otsu法阈值分割算法
def YCrCb_OTSU(img):
    start = time.time()
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像

    # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # 对cr通道分量进行高斯滤波
    # 根据OTSU算法求图像阈值, 对图像进行二值化
    th, skin = cv2.threshold(cr1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    end = time.time()
    print("阈值为"+str(th))
    print("耗时"+str(end-start)+"秒")
    cv2.imshow("image CR", cr1)
    cv2.imshow("Skin Cr+OSTU", skin )
    cv2.waitKey(0)
    return skin


YCrCb_OTSU(Img1)

In [ ]:
# 绘制YCrCb颜色空间图像
ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  
(y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像

fig, axs = plt.subplots(1, 4, figsize=(10,10))          # 创建1X4的网格，其中每个子图大小为7X7 英寸：

plt.subplot(1,4,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("原图")                                       # 为第一个子图添加标题
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,2)
plt.imshow(y,cmap='Greys_r')
plt.title("y分量")                                       
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,3)
plt.imshow(cr,cmap='Greys_r')
plt.title("Cr分量")                                       
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show

plt.subplot(1,4,4)
plt.imshow(cb,cmap='Greys_r')
plt.title("Cb分量")  
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.show



In [ ]:
# 像素分布概率直方图
def showimg_pro_eachgray(img):                                          # 显示每一个灰度级的像素数概率
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                          # 灰度化处理
          
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])            # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                          # 获得每个灰度级中像素数占总像素数比例则我们获得了pi的一个向量
    x = np.linspace(0, 255, 256)                                        # 横坐标灰度级别
    plt.subplot(2,1,1)
    plt.imshow(img,'gray')                                              # 如果不加gray就会出现以前hsv与rgb的问题
    plt.title('origin image')
    plt.colorbar(orientation="horizontal")                              # 水平放置颜色条
    plt.subplot(2,1,2)
    plt.bar(x, p.ravel(), 0.9, alpha=1, color='b')
    plt.title('Histogram of the probablity for each gray level',y=-0.4) # 调节标题上下移动的方法
    plt.show()

showimg_pro_eachgray(img)


In [ ]:
# 手撸实现OTSU
def OTSU(img):
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    maxvar=0
    mG = np.sum(x*p)                                                        # 平均灰度级
    th=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        p1 = np.sum(p[0:k1])
        p2 = np.sum(p[k1:256])
        m1 = np.dot(x[0:k1],p[0:k1])/p1
        m2 = np.dot(x[k1:256],p[k1:256])/p2
        vars = p1*p2*(m1-m2)**2
        if(vars>maxvar):
            maxvar = vars
            th = k1
            step = step+1
            print("第"+str(step)+"次找到最佳阈值"+str(th)+","+"最大类间方差为"+str(maxvar))
    return th,maxvar 
print("最佳阈值是"+str(OTSU(img)[0]),"最大方差为"+str(OTSU(img)[1]))


# 基于手撸的OTSU 实现的YCrCb分割                  # 凸显改进算法的优越性，使用mopi.jpeg例子，能良好体现出改进之后的算法，具有更强的识别效果，在处理肤色相近，像素值集中的图像时更占上风
def YCrCb_OTSU(img):
    start  = time.time()                               # 引入计时模块
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  有没有转换公式呢？
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    th = OTSU(cr1)[0]
    skin = cr1.copy()
    # 根据OTSU算法求图像阈值, 对图像进行二值化 获得人脸部分（白色）,会发现因为手部与脸部肤色相近，故被认定为前景（人脸部分）
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if(cr1[i,j]< th):           
                skin[i,j]= 0  
            else:  
                skin[i,j] = 255

    # # 原图的人脸部分
    # for i in range(0,img.shape[ßß0]):
    #     for j in range(0,img.shape[1]): 
    #         if(skin[i,j]==0):
    #             img[i,j] = 0
    # cv2.imshow("SKin",img)
    end = time.time()
    print("耗时"+str(end-start)+"s")
    # 显示图像
    cv2.imshow("Skin Cr+OSTU_Hand", skin) # 因为通过对YCrCb空间分割再还原RGB比较麻烦
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#YCrCb_OTSU(img)

In [ ]:
# OTSU法实现图像分割
def OTSU_Seg(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    _, obj = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # cv2.imshow("img",img)
    # cv2.imshow("Use_OTSU_Seg",obj)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return obj

obj = OTSU_Seg(img)  
#figure()
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Lena.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("调用threshold函数+OTSU阈值分割后")                              # 为第一个子图添加标题


In [ ]:
# 手撸OTSU实现图像分割
def OTSU_Seg_Hand(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    th = OTSU(img)[0]
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):     
            if(img[i,j]<th):
                img[i,j]=0
            else:
                img[i,j]=255
    return img 
obj = OTSU_Seg_Hand(img)  
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Cameraman.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("使用手撸OTSU法阈值分割后")                              # 为第一个子图添加标题

In [ ]:
# 多阈值分割的OTSU改进算法    用于处理口红色号较深，将口红认定为前景的情况
def OTSU_Mult_IMPROVE(img):
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    sigma_b = 0                                                             # 类间方差
    sigma_in = 0                                                            # 类内方差
    maxvar = 0                                                              # 最大类间类内方差
    mu_T = np.sum(x*p)                                                        # 平均灰度级
    th1=0
    th2=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        for k2 in range(1,256):
                a0 = np.sum(p[0:k1])
                a1 = np.sum(p[k1:256])
                a2 = 1-a0-a1
                mu_0 = np.dot(x[0:k1],p[0:k1])/a0
                mu_1 = np.dot(x[k1:k2],p[k1:k2])/a1
                mu_2 = np.dot(x[k2:256],p[k2:256])/a2
                vars1 = a0*(mu_0-mu_T)**2+a1*(mu_1-mu_T)**2+a2*(mu_2-mu_T)**2

    return th1,th2,maxvar


In [ ]:
# 基于手撸的OTSU_IMPROVE 实现的YCrCb分割                  
# 凸显改进算法的优越性，使用mopi.jpeg例子，能良好体现出改进之后的算法，具有更强的识别效果，在处理肤色相近，像素值集中的图像时更占上风

#  手撸实现最大类间类内方差比 
def OTSU_IMPROVE(img):
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                          # 若待划分的图像是单通道，则无需转为灰度图
    pixels = cv2.calcHist([img], [0], None, [256], [0, 256])                # 计算每个灰度级中所含像素数，返回的是一个（256,1）的数组
    p = pixels / (img.shape[0] * img.shape[1])                              # 获得每个灰度级中像素数占总像素数比例则我们获得了p_i的一个向量
    x=np.linspace(1,256,256)                                                # 灰度级像素范围定义，若定义为0～255，则求平均灰度级时会忽略第一个数据，所以我们定义为从1到256                                                 
    sigma_b = 0                                                             # 类间方差
    sigma_in = 0                                                            # 类内方差
    maxvar = 0                                                              # 最大类间类内方差比
    mG = np.sum(x*p)                                                        # 平均灰度级
    th=0
    step = 0
    for k1 in range(1,256):                                                 # 暴力搜索
        p1 = np.sum(p[0:k1])
        p2 = np.sum(p[k1:256])
        m1 = np.dot(x[0:k1],p[0:k1])/p1
        m2 = np.dot(x[k1:256],p[k1:256])/p2
        vars1 = p1*p2*(m1-m2)**2
        if(vars1>sigma_b):
            sigma_b = vars1
        vars2 = np.sum(p[0:k1]*(x[0:k1]-m1)**2)+np.sum(p[k1:256]*(x[k1:256]-m2)**2)
        if(vars2>sigma_in):
            sigma_in = vars2
        if(sigma_in>0):
            vars = sigma_b/sigma_in
        else:
            vars = 0
        if(vars>maxvar):
            maxvar = vars
            th = k1
            step = step + 1
            print("第"+str(step)+"次找到最佳阈值"+str(th)+","+"最大类间方差为"+str(maxvar))
    return th,maxvar 

print("最佳阈值是"+str(OTSU_IMPROVE(img)[0]),"最大方差比为"+str(OTSU_IMPROVE(img)[1]))    

        
def YCrCb_OTSU_IMPROVE(img):
    start  = time.time()                               # 引入计时模块
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)      #   RGB->YCrCb  有没有转换公式呢？
    (y,cr,cb) = cv2.split(ycrcb)                       # 图像分割，分别获取y,cr,br通道图像
    cr1 = cv2.GaussianBlur(cr, (5, 5), 0) # # 高斯滤波, cr 是待滤波的源图像数据, (5,5)是值窗口大小, 0 是指根据窗口大小来计算高斯函数标准差
    th = OTSU_IMPROVE(cr1)[0]
    skin = cr1.copy()
    # 根据OTSU算法求图像阈值, 对图像进行二值化 获得人脸部分（白色）,会发现因为手部与脸部肤色相近，故被认定为前景（人脸部分）
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if(cr1[i,j]< th):           
                skin[i,j]= 0  
            else:  
                skin[i,j] = 255

    # # 原图的人脸部分
    # for i in range(0,img.shape[ßß0]):
    #     for j in range(0,img.shape[1]): 
    #         if(skin[i,j]==0):
    #             img[i,j] = 0
    # cv2.imshow("SKin",img)
    end = time.time()
    print("耗时"+str(end-start)+"s")
    #显示图像
    cv2.imshow("Skin Cr+OSTU_IMPROVE_Hand", skin) # 因为通过对YCrCb空间分割再还原RGB比较麻烦
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin

YCrCb_OTSU_IMPROVE(img)




In [ ]:
# YCrCb_OTSU及其改进算法的作图
skin1 = YCrCb_OTSU(img)
skin2=  YCrCb_OTSU_IMPROVE(img)
fig, axs = plt.subplots(1, 3, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,3,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("原图")                                       # 为第一个子图添加标题

plt.subplot(1,3,2)                                      # 占据第二个位置
plt.imshow(skin1,cmap='Greys_r')                         
plt.title("使用手撸OTSU法阈值分割后")                          

plt.subplot(1,3,3)                                      # 占据第三个位置
plt.imshow(skin2,cmap='Greys_r')                        
plt.title("使用YCrCb+改进OTSU法阈值分割后")


In [ ]:
# 手撸OTSU_IMPROVE实现图像分割
def OTSU_Seg_IMPROVE(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)                             # 若待划分的图像是单通道，则无需转为灰度图
    th = OTSU_IMPROVE(img)[0]
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):     
            if(img[i,j]<th):
                img[i,j]=0
            else:
                img[i,j]=255
    return img 
obj = OTSU_Seg_IMPROVE(img)  
fig, axs = plt.subplots(1, 2, figsize=(14,14))          # 创建1X2的网格，其中每个子图大小为14X14 英寸：
plt.subplot(1,2,1)                                      # 占据第一个位置
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           # opencv读取的图像是BGR通道的，plt输出时是RGB通道
plt.imshow(img_RGB)
plt.title("Cameraman.jpg")                                       # 为第一个子图添加标题

plt.subplot(1,2,2)                                      # 占据第二个位置
plt.imshow(obj,cmap='Greys_r')                          # plt默认显示三通道图像
plt.title("使用改进的OTSU法阈值分割后")                              # 为第一个子图添加标题

In [ ]:
# 二次多项式模式检测
def Quad_poly(img):
    Info = img.shape
    height = Info[0]
    width = Info[1]
    Channels = Info[2]
    # 创建一个与原图像大小相同的零数组
    skin = np.zeros((height,width),np.uint8)
    for i in range(0,height):
        for j in range(0,width):
            B = img[i,j,0]
            G = img[i,j,1]
            R = img[i,j,2]
            if(R-G>=20):
                if(G>B):
                    sum = R+G+B
                    t1 = 100*R-33*sum
                    t2 = 100*G-33*sum
                    if((t1**2+t2**2)>=4*sum*sum):
                        T1 = 10000*G*sum
                        lower = -7760*R**2 + 5601*R*sum + 1766*sum**2
                        if(T1>lower):
                            upper = -13767*R**2 + 10743*R*sum + 1452*sum**2
                            if(T1<upper):
                                skin[i,j] = 255
    cv2.imshow('skin2',skin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin
#skin = Quad_poly(img)
# 效果垃圾 笑果还行     人脸面部会反光，导致反光区域的RGB值偏高，会被筛选出人脸区域，而人脸轮廓例如脸颊、腮部亮度比较稳定，会被识别为人脸区域


In [ ]:
# 基于HSV颜色空间H，S，V范围筛选法
def HSV(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 把图像转换到HSV色域
    (h, s, v) = cv2.split(hsv) # 图像分割, 分别获取h, s, v 通道分量图像
    skin = np.zeros(h.shape, np.uint8)  # 根据源图像的大小创建一个全0的矩阵,用于保存图像数据
    (height, width) = img.shape[0:2] # 获取源图像数据的长和宽

    # 遍历图像, 判断HSV通道的数值, 如果在指定范围中, 则置把新图像的点设为255,否则设为0
    for i in  range(0, height):
        for j in  range(0, width):
            if (h[i][j] >  2) and (h[i][j] <  15) and (s[i][j] >  35) and (s[i][j] <  255) and (v[i][j] >  60) and (v[i][j] <  255):
                skin[i][j] =  255
            else:
                skin[i][j] =  0
    cv2.imshow('skin',skin)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return skin

# hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # 把图像转换到HSV色域
# cv2.imshow('hsv',hsv)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
#HSV(img)
    


In [ ]:
# 双边滤波
dst1 = cv2.bilateralFilter(img,15,35,35) # 邻域半径为15 sigma_d为空间高斯函数标准差，sigma_r为灰度值相似性高斯函数标准差
#cv2.imshow('dst1',dst1)


In [ ]:
# 磨皮算法 
# 通过上述算法获得皮肤区域，遍历双边滤波后的矩阵，将非皮肤区域还原
def skin_(dst1,img,skin1):       #dst1 为双边滤波后的矩阵  img 为原图 skin1 为皮肤区域
    dst2 = dst1.copy()
    for i in range(0,height):   
        for j in range(0,width):
            if(skin1[i,j] == 0):
                dst2[i,j] = img[i,j]

    cv2.imshow('dst2',dst2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()